In [ ]:
import pandas as pd
import numpy as np

import lightgbm as lgb
import catboost as cbt

import datetime

from sklearn.preprocessing import LabelEncoder
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import shap

In [ ]:
import sys
sys.path.insert(0, 'old_roman/IEEE_FRAUD/')

In [ ]:

from generate_features import generate_features_time_series

In [ ]:
%%time
folder_path = 'data/'
train = pd.read_pickle(f'{folder_path}train.pkl')
test = pd.read_pickle(f'{folder_path}test.pkl')

In [ ]:
%%time
df, enc = generate_features_time_series(train, test)

In [ ]:
from settings import CATEGORICAL_FEATURES, COLUMNS_TO_REMOVE

In [ ]:
COLUMNS_TO_REMOVE

In [ ]:
df.shape

In [ ]:
df.drop(['TransactionID',
 'TransactionDT',
 'isFraud',
 'TransactionDT_split',
 'TransactionDT_year',
 'TransactionDT_month'], axis=1, inplace=True)

In [ ]:
default_cols = set(test.columns).intersection(df.columns)

In [ ]:
df.drop(['index'] + list(default_cols), axis=1, inplace=True)

In [ ]:
df.drop(['TransactionDT_dayOfMonth',
 'TransactionDT_weekOfMonth',
 'TransactionDT_hour', 'is_foreign', 'cents', 'cents_categorical'], axis=1, inplace=True)

In [ ]:
df.to_pickle('subcard_strikes_back_features.pkl')

In [ ]:
df.shape

In [ ]:
identity_cols = !head -1 data/train_identity.csv
identity_cols = identity_cols[0].split(',')

In [ ]:
N_TRAIN_EXAMPLES = len(train)

In [ ]:
tr_am = pd.read_csv(f'{folder_path}train_transaction.csv', usecols=['TransactionAmt'], dtype=str)
te_am = pd.read_csv(f'{folder_path}test_transaction.csv', usecols=['TransactionAmt'], dtype=str)
train[['dollars', 'cents']] = tr_am.TransactionAmt.str.split('.', expand=True).astype(int)
test[['dollars', 'cents']] = te_am.TransactionAmt.str.split('.', expand=True).astype(int)

In [ ]:
roman_df = pd.read_pickle('data/roman_features.pkl')

In [ ]:
roman_df.reset_index(drop=True, inplace=True)
roman_df.set_index('index', inplace=True)

In [ ]:
roman_df.drop(labels=['isFraud', 'is_foreign', 'is_holiday'], inplace=True, axis=1)

In [ ]:
roman_feature_names = set(roman_df.columns.to_list())

In [ ]:
MODEL_FEATURES = set(test.columns) - set(['TransactionDT', 'TransactionID'])

In [ ]:
basic_features = set(test.columns) - set(['TransactionDT', 'TransactionID'])

In [ ]:
CATEGORICAL_FEATURES =  set(['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
            'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37',
            'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 
            'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9'])

Говорят, что неплохо бы дропнуть те карты, которые не встречается или в трейне, или в тесте

In [ ]:
for col in ['card1']: 
    valid_card = pd.concat([train[[col]], test[[col]]])
    valid_card_std = valid_card.values.std()

    invalid_cards = valid_card[valid_card <= 2]
    print('Rare cards',len(invalid_cards))

    valid_card = valid_card[valid_card > 2]
    valid_card = list(valid_card.index)

    print('No intersection in Train', len(train[~train[col].isin(test[col])]))
    print('Intersection in Train', len(train[train[col].isin(test[col])]))
    
    train[col] = np.where(train[col].isin(test[col]), train[col], np.nan)
    test[col]  = np.where(test[col].isin(train[col]), test[col], np.nan)

    train[col] = np.where(train[col].isin(valid_card), train[col], np.nan)
    test[col]  = np.where(test[col].isin(valid_card), test[col], np.nan)
    print('#'*20)

for col in ['card2','card3','card4','card5','card6',]: 
    print('No intersection in Train', col, len(train[~train[col].isin(test[col])]))
    print('Intersection in Train', col, len(train[train[col].isin(test[col])]))
    
    train[col] = np.where(train[col].isin(test[col]), train[col], np.nan)
    test[col]  = np.where(test[col].isin(train[col]), test[col], np.nan)
    print('#'*20)

In [ ]:
all_df = pd.concat([train, test])
all_df = pd.concat([all_df, roman_df], axis=1)

# Фичи по датам

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

dates_range = pd.date_range(start='2017-10-01', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

In [ ]:
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')


In [ ]:
del train, test, roman_df

In [ ]:
import gc

In [ ]:
gc.collect()

In [ ]:
def datetime_features(all_df):
    all_df['DT'] = all_df['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
    all_df['dayofweek'] = all_df['DT'].dt.dayofweek
    all_df['dayofmonth'] = all_df['DT'].dt.day
    all_df['hour'] = all_df['DT'].dt.hour
    all_df['weekofmonth'] = (all_df['DT'].dt.day - 1) // 7 + 1
    #all_df['is_holiday'] = (all_df['DT'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
    new_features = ['dayofweek', 'dayofmonth', 'hour', 'weekofmonth']#, 'is_holiday']
    return new_features, new_features

In [ ]:
%%time
a, c = datetime_features(all_df)
MODEL_FEATURES.update(a)
CATEGORICAL_FEATURES.update(c)

In [ ]:
all_df['DT_split'] = (all_df['DT'].dt.year - 2017) * 12 + all_df['DT'].dt.month
N_TRAIN = sum(all_df['DT_split'] < 17)

In [ ]:
all_df['day'] = ((all_df['DT'].dt.year-2017)*365 + all_df['DT'].dt.dayofyear).astype(np.int16)
all_df['hour_day'] = all_df['DT'].dt.hour + 24 * (all_df['day'] - 335)

# D-features

In [ ]:
d_cols = [col for col in all_df.columns if col.startswith('D') and not col[-1:].isalpha() and col != 'D9']

In [ ]:
d_cols_notnull = [d + '_notnull' for d in d_cols + ['D9']]
all_df[d_cols_notnull] = all_df[d_cols + ['D9']].notnull()

In [ ]:
%%time
from itertools import combinations
versus_cols = []
for a, b in combinations(d_cols, 2):
    col = a + '_vs_' + b
    versus_cols.append(col)
    # как, блин, оставить нан в сравнениях?
    all_df[col] = (all_df[a] == all_df[b]) + all_df[a] - all_df[a] + all_df[b] - all_df[b]

In [ ]:
# Lets transform D8 and D9 column
# As we almost sure it has connection with hours
all_df['D8_not_same_day'] = np.where(all_df['D8']>=1,1,0)
all_df['D8_D9_decimal_dist'] = all_df['D8'].fillna(0)-all_df['D8'].fillna(0).astype(int)
all_df['D8_D9_decimal_dist'] = ((all_df['D8_D9_decimal_dist']-all_df['D9'])**2)**0.5
all_df['D8'] = all_df['D8'].fillna(-1).astype(int)

In [ ]:
MODEL_FEATURES.update(['D8_not_same_day', 'D8_D9_decimal_dist', 'D8_D9_decimal_dist'])

In [ ]:
def values_normalization(all_df, period, col, clip=True, minmax=True):
        new_col = col + '_' + period
        df = all_df[[col, period]].copy()
        df[col] = df[col].astype(float)
        if clip:
            df[col] = df[col].clip(0) 

        aggs = df.groupby([period])[col].agg(['min', 'max', 'std', 'mean'])
        
        agg_max = aggs['max'].to_dict()
        agg_min = aggs['min'].to_dict()
        agg_std = aggs['std'].to_dict()
        agg_mean = aggs['mean'].to_dict()

        all_df['temp_min'] = all_df[period].map(agg_max)
        all_df['temp_max'] = all_df[period].map(agg_min)
        all_df['temp_std'] = all_df[period].map(agg_std)
        all_df['temp_mean'] = all_df[period].map(agg_mean)
        

        all_df[new_col + '_min_max'] = ((all_df[col] - all_df['temp_min']) /\
                (all_df['temp_max'] - all_df['temp_min'])).astype(float)
        
        all_df[new_col + '_std_score'] = (all_df[col] - all_df['temp_mean']) / (all_df['temp_std'])

        del all_df['temp_min'], all_df['temp_max'], all_df['temp_std'], all_df['temp_mean']

In [ ]:
%%time
for period in ['day']:
    for col in d_cols:
        values_normalization(all_df, period, col, minmax=True)

In [ ]:
for col in ['D1','D2']:
    all_df[col + '_scaled'] = all_df[col] / all_df[:N_TRAIN_EXAMPLES][col].max()

In [ ]:
MODEL_FEATURES.update(['D1_scaled', 'D2_scaled'])

# E-mail features

In [ ]:
def mail_split_features(df):
    p_domain = ['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']
    r_domain = ['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
    all_df[p_domain] =\
        all_df['P_emaildomain'].str.split('.', expand=True)
    all_df[r_domain] =\
        all_df['P_emaildomain'].str.split('.', expand=True)
    return p_domain + r_domain, p_domain + r_domain
    

In [ ]:
%%time
a, c = mail_split_features(all_df)
MODEL_FEATURES.update(a)
CATEGORICAL_FEATURES.update(c)

# Device info

In [ ]:
all_df['DeviceInfo'].fillna('', inplace=True)
all_df['id_30'].fillna('', inplace=True)
all_df['id_31'].fillna('', inplace=True)

In [ ]:
def add_device_features(all_df):
    all_df['DeviceInfoMajor'] = all_df['DeviceInfo'].str.split(' ', expand=True)[0]
    all_df['DeviceInfoTop'] = all_df['DeviceInfo'].str.split('-', expand=True)[0]
    all_df['DeviceInfoIsRV'] = all_df['DeviceInfoMajor'].apply(lambda x: 'rv' in x)
    return ['DeviceInfoMajor', 'DeviceInfoTop', 'DeviceInfoIsRV'], ['DeviceInfoMajor', 'DeviceInfoTop']

In [ ]:
%%time
a, c = add_device_features(all_df)
MODEL_FEATURES.update(a)
CATEGORICAL_FEATURES.update(c)

In [ ]:
def split_version(version):
    n_sep_a = version.count('.')
    n_sep_b = version.count('_')
    if n_sep_a > 0:
        return version.split('.')[0]
    elif n_sep_b > 0:
        return version.split('_')[0]
    else:
        return version

def split_os(os):
    spl = os.split(' ')
    if len(spl) > 1:
        os_name = ' '.join(spl[:-1])
        major_version = split_version(spl[-1])
        return os_name, ' '.join([os_name, major_version])
    else:
        return os, ''
# TODO: add minor

In [ ]:
def add_os_features(all_df):
    os, version = zip(*all_df['id_30'].apply(lambda x: split_os(x)).values)
    all_df['OSName'] = os
    all_df['OSMajorVersion'] = version
    return ['OSName', 'OSMajorVersion'], ['OSName', 'OSMajorVersion']

In [ ]:
%%time
a, c = add_os_features(all_df)
MODEL_FEATURES.update(a)
CATEGORICAL_FEATURES.update(c)

In [ ]:
# SHAME ON ME
def get_browser(browser):
    if 'safari' in browser:
        return 'safari'
    
    if 'chrome' in browser:
        return 'chrome'
    
    if browser.startswith('ie'):
        return 'internetexplorer'
    
    if 'edge' in browser:
        return 'edge'
    
    if 'firefox' in browser.lower():
        return 'firefox'
    
    if 'samsung' in browser.lower():
        return 'samsung'
    
    if 'google' in browser:
        return 'google'
    
    if 'opera' in browser:
        return 'opera'
    
    if 'android' in browser.lower():
        return 'android'
    
    return browser

def is_mobile(browser):
    br = browser.lower()
    if 'mobile' in br or 'for android' in br:
        return True
    else:
        return False


In [ ]:
os_data = pd.read_csv('os_releases.csv')
browser_data = pd.read_csv('browser_releases.csv')

In [ ]:
def get_browser_features(all_df):
    all_df['Browser'] = all_df['id_31'].apply(lambda x: get_browser(x))
    all_df['IsMobile'] = all_df['id_31'].apply(lambda x: is_mobile(x))
    all_df['VersionNum'] = all_df['id_31'].fillna('0').\
        apply(lambda x: ''.join([i for i in x if i.isnumeric()]))
    #all_df['BrowserVersion'] = all_df['Browser'] + ' ' + all_df['VersionNum'].astype('str')
    
    return ['Browser', 'IsMobile', 'VersionNum'],  ['Browser', 'IsMobile']

In [ ]:
%%time
a, c = get_browser_features(all_df)
MODEL_FEATURES.update(a)
CATEGORICAL_FEATURES.update(c)

# Some with M

In [ ]:
########################### M columns (except M4)
# All these columns are binary encoded 1/0
# We can have some features from it
i_cols = ['M1','M2','M3','M5','M6','M7','M8','M9']

all_df['M_sum'] = all_df[i_cols].sum(axis=1).astype(np.int8)
all_df['M_na'] = all_df[i_cols].isna().sum(axis=1).astype(np.int8)

MODEL_FEATURES.update(['M_sum', 'M_na'])

# NaN count

In [ ]:
all_df['notnull_count'] = all_df[basic_features].notnull().sum(axis=1)

In [ ]:
MODEL_FEATURES.add('notnull_count')

# UID

In [ ]:
cols_for_uid = ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']

In [ ]:
all_df[cols_for_uid] = all_df[cols_for_uid].astype(str)

In [ ]:
all_df['uid1'] = all_df['card1'] + '_' + all_df['card2']

all_df['uid2'] = all_df['uid1'] + '_' + all_df['card3'] + '_' + all_df['card5']

all_df['uid3'] = all_df['uid2'] + '_' + all_df['addr1'] + '_' + all_df['addr2']

all_df['uid4'] = all_df['uid3'] + '_' + all_df['P_emaildomain']

all_df['uid5'] = all_df['uid3'] + '_' + all_df['R_emaildomain']

In [ ]:
uids = [f'uid{i}' for i in range(1, 6)]

 # TransactionAmt features

Говорят, если в сумме транзакции есть более двух чисел после запятой, то это транзакция в иностранной валюте

In [ ]:
all_df['is_foreign'] = all_df['cents'].apply(lambda x: len(str(x)) > 2)

In [ ]:
MODEL_FEATURES.add('is_foreign')

А раз уникальных TransactionAmt не очень много, то почему бы не добавить nunique для каждого из айдишников сверху

In [ ]:
all_df['TransactionAmt'].nunique()

In [ ]:
from sklearn.preprocessing import QuantileTransformer
from collections import Counter

In [ ]:
%%time
unique_amt_cols = []
for uid in uids:
    uniques = all_df.groupby(uid)['TransactionAmt'].nunique()
    all_df[uid + '_unique_amt'] = all_df[uid].map(uniques)
    unique_amt_cols.append(uid + '_unique_amt')
    
QT = QuantileTransformer(n_quantiles=500)
all_df[unique_amt_cols] = QT.fit_transform(all_df[unique_amt_cols])

In [ ]:
MODEL_FEATURES.update(unique_amt_cols)

In [ ]:
%%time
for period in ['day']:
    values_normalization(all_df, period, 'TransactionAmt', minmax=True)

In [ ]:
transaction_norm = ['TransactionAmt_day_min_max', 'TransactionAmt_hour_day_min_max'] + ['TransactionAmt_day_std_score', 'TransactionAmt_hour_day_std_score']

In [ ]:
MODEL_FEATURES.update(['TransactionAmt_day_min_max', 'TransactionAmt_hour_day_min_max'])
MODEL_FEATURES.update(['TransactionAmt_day_std_score', 'TransactionAmt_hour_day_std_score'])

# Следующая транзакция

In [ ]:
all_df.set_index('card1', inplace=True)

In [ ]:
%%time
same_cols = []
for i in range(1, 4):
    a = 'is_same_next_transaction_' + str(i)
    b = 'is_same_prev_transaction_' + str(i)
    all_df[a] = all_df.groupby('card1')['TransactionAmt'].diff(i) == 0
    all_df[b] = all_df.groupby('card1')['TransactionAmt'].diff(-i) == 0
    same_cols.extend([a, b])

In [ ]:
all_df['same_transaction_near'] = all_df[same_cols].sum(axis=1)

In [ ]:
MODEL_FEATURES.add('same_transaction_near')

In [ ]:
all_df.drop(same_cols, axis=1, inplace=True)

In [ ]:
all_df.reset_index(inplace=True)

In [ ]:
is_same_cols = ['is_same_next_transaction', 'is_same_prev_transaction']

# Frequency Encoding

In [ ]:
def encode_frequency(col, quantile=True):
    return col.map(col.value_counts().to_dict())

In [ ]:
%%time
freq_cols = []
for col in list(CATEGORICAL_FEATURES) + uids + ['cents', 'dollars']:
    all_df[col + '_freq'] = encode_frequency(all_df[col])
    freq_cols.append(col + '_freq')
    
QT = QuantileTransformer(n_quantiles=500)
all_df[freq_cols] = QT.fit_transform(all_df[freq_cols])

In [ ]:
MODEL_FEATURES.update(freq_cols)

# Categorical

In [ ]:
from multiprocessing import Pool

In [ ]:
def encode(col):
    le = LabelEncoder()
    le.fit(all_df[col].astype(str).values)
    return le.transform(all_df[col].astype(str).values)

In [ ]:
all_df['cents_categorical'] = all_df['cents'].copy()
all_df['dollars_categorical'] = all_df['dollars'].copy()

In [ ]:
CATEGORICAL_FEATURES.update(['cents_categorical', 'dollars_categorical'])

In [ ]:
MODEL_FEATURES.update(['cents_categorical', 'dollars_categorical'])

In [ ]:
%%time
with Pool(16) as pool:
    encodes = pool.map(encode, CATEGORICAL_FEATURES)

for name, enc in zip(CATEGORICAL_FEATURES, encodes):
    all_df[name] = enc

# Numerical encoding

In [ ]:
def calc_smooth_encoding(all_df, by, on, m):
    # Compute the global mean
    mean = all_df[on].mean()
    
    std = all_df[on].std()
    
    median = np.nanmedian(all_df[on])

    # Compute the number of values and the mean of each group
    agg = all_df.groupby(by)[on].agg(['count', 'mean', 'std', np.nanmedian])
    counts = agg['count']
    means = agg['mean']
    stds = agg['std']
    medians = agg['nanmedian']
    

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)
    
    smooth_std = (stds * counts + m * std ) / (counts + m)
    
    smooth_median = (medians * counts + m * median ) / (counts + m)

    # Replace each value by the according smoothed mean
    return all_df[by].map(smooth), all_df[by].map(smooth_std), all_df[by].map(smooth_median)

In [ ]:
%%time
amt_features = []
for col in uids:
    mean, std, median = calc_smooth_encoding(all_df, col, 'TransactionAmt', 30)
    all_df[col + '_TransactionAmt_mean'] = mean
    all_df[col + '_TransactionAmt_std'] = std
    #all_df[col + '_TransactionAmt_median'] = median
    amt_features.extend([col + '_TransactionAmt_mean', col + '_TransactionAmt_std'])#, col + '_TransactionAmt_median'])

In [ ]:
%%time
D15_features = []
for col in uids:
    mean, std, median = calc_smooth_encoding(all_df, col, 'D15_day_min_max', 30)
    all_df[col + '_D15_mean'] = mean
    all_df[col + '_D15_std'] = std
    #all_df[col + '_D15_median'] = median
    D15_features.extend([col + '_D15_mean', col + '_D15_std'])#, col + '_D15_median'])

In [ ]:
%%time
C13_features = []
for col in uids:
    mean, std, median = calc_smooth_encoding(all_df, col, 'C13', 30)
    all_df[col + '_C13_mean'] = mean
    all_df[col + '_C13_std'] = std
    #all_df[col + '_C13_median'] = median
    C13_features.extend([col + '_C13_mean', col + '_C13_std'])#, col + '_C13_median']

In [ ]:
MODEL_FEATURES.update(D15_features + C13_features)

In [ ]:
MODEL_FEATURES.update(amt_features)

In [ ]:
all_df.to_pickle('all_my_features.pkl')

# Splits

In [ ]:
train_final = all_df[:N_TRAIN_EXAMPLES]
test_final = all_df[N_TRAIN_EXAMPLES:]

tr = train_final[:N_TRAIN]
val = train_final[N_TRAIN:]

In [ ]:
def downsample(df, how_strong=2):
    positive = df[df.isFraud == 1]
    negative = df[df.isFraud == 0]
    negative = negative.sample(int(len(negative) / how_strong))
    res = pd.concat([positive, negative])
    return res.sample(len(res))

In [ ]:
tr = downsample(tr, 5)
val = downsample(val, 5)

# Train

In [ ]:
add_cols = [col + '_day_min_max' for col in d_cols] 
add_cols = add_cols + [col + '_day_std_score' for col in d_cols]

In [ ]:
import pickle

In [ ]:
with open('col_names.pkl', 'wb') as f:
    pickle.dump((MODEL_FEATURES, CATEGORICAL_FEATURES, roman_feature_names, roman_categorical), f)

In [ ]:
from IEEE_FRAUD.settings import CATEGORICAL_FEATURES as roman_categorical

In [ ]:
good_cols = list(MODEL_FEATURES - set(['VersionNum']) - set(['card1']) -\
                    set(d_cols)) + add_cols
good_categotical = list(set(CATEGORICAL_FEATURES) - set(['card1']))

In [ ]:
all_good_features = set(good_cols + list(roman_feature_names)) - set(['card1', 'dayofmonth'])
all_good_categorical = set(good_categotical + roman_categorical) - set(['card1', 'is_holiday', 'dayofmonth'])

In [ ]:
dtrain = lgb.Dataset(tr[all_good_features], label=tr['isFraud'],
                     categorical_feature=all_good_categorical,free_raw_data=False)
dval = lgb.Dataset(val[all_good_features], label=val['isFraud'],
                   categorical_feature=all_good_categorical, free_raw_data=False)

In [ ]:
params = {'num_leaves': 200,
          'min_child_samples': 79,
          #'min_sum_hessian_in_leaf': 5e-3,
          #'max_bin': 511,
          #'min_data_in_leaf': 
          #'scale_pos_weight': 2,
          'objective': 'binary',
          'metric': 'auc',
          'max_depth': 13,
          'learning_rate': 0.01,
          "boosting_type": "goss",
          "bagging_freq": 0,
          'nthread': 16,
          
          'pos_bagging_fraction': 0.8,
          'neg_bagging_fraction': 0.01,
          "bagging_fraction": 0.9,
          "bagging_seed": 11,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          
          'feature_fraction': 0.9,
          'min_data_per_group': 50,
          'cat_smooth': 500,
          'max_cat_to_onehot': 8
          #'categorical_feature': cat_cols
         }

In [ ]:
%%time
model = lgb.train(params, dtrain, num_boost_round=10000,
                  valid_sets=(dval, dtrain), valid_names=('val', 'train'),
                early_stopping_rounds=25, verbose_eval=25)

In [ ]:
%%time
model = lgb.train(params, dtrain, num_boost_round=10000,
                  valid_sets=(dval, dtrain), valid_names=('val', 'train'),
                early_stopping_rounds=25, verbose_eval=25)

# Split importance

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(model.feature_importance('split'), good_cols)), columns=['Value','Feature'])
bad_cols = feature_imp[feature_imp['Value'] < 1]['Feature']
len(bad_cols)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(model.feature_importance('split'), all_good_features)), columns=['Value','Feature'])
bad_cols = feature_imp[feature_imp['Value'] < 1]['Feature']
plt.figure(figsize=(10, 20))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:100])
plt.tight_layout()
plt.show()

# SHAP importance

In [ ]:
shap_test = test_final.sample(10000)

In [ ]:
shap_imp[shap_imp['Feature'] == 'cents_freq']

In [ ]:
%%time
res = model.predict(shap_test[all_good_features], pred_contrib=True)
mean_shap = np.abs(res).mean(axis=0)
shap_imp = pd.DataFrame(sorted(zip(mean_shap, all_good_features)), columns=['Value','Feature'])
plt.figure(figsize=(10, 10))
sns.barplot(x="Value", y="Feature", data=shap_imp.sort_values(by="Value", ascending=False)[:60])
plt.tight_layout()
plt.show()

# Fit KFold (Scary)

In [ ]:
from sklearn.model_selection import GroupKFold

In [ ]:
gkf = GroupKFold(6)

In [ ]:
good_cols_kf = list(set(good_cols) - set(bad_cols))
good_categorical_kf = list(set(good_categotical) - set(bad_cols))

In [ ]:
train_final['SPLITER'] = ((train_final['DT'].dt.month * 30 + train_final['DT'].dt.day) // 15).copy()

In [ ]:
train_final['DT_split'].value_counts()

In [ ]:
groups = ((train_final['DT'].dt.dayofyear + 31) % 365) // 15

In [ ]:
groups[groups == 12] = 11

In [ ]:
groups.value_counts()

In [ ]:
params = {'num_leaves': 200,
          'min_child_samples': 79,
          #'min_sum_hessian_in_leaf': 5e-3,
          #'max_bin': 511,
          #'min_data_in_leaf': 
          #'scale_pos_weight': 2,
          'objective': 'binary',
          'metric': 'auc',
          'max_depth': 13,
          'learning_rate': 0.01,
          "boosting_type": "goss",
          "bagging_freq": 0,
          'nthread': 16,
          
          'pos_bagging_fraction': 0.8,
          'neg_bagging_fraction': 0.01,
          "bagging_fraction": 0.9,
          "bagging_seed": 11,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          
          'feature_fraction': 0.9,
          'min_data_per_group': 50,
          'cat_smooth': 500,
          'max_cat_to_onehot': 8
          #'categorical_feature': cat_cols
         }

In [ ]:
import gc

In [ ]:
del dtrain, dval

In [ ]:
gc.collect()

In [ ]:
scores = []
all_preds = np.zeros(len(test_final))
for train_ids, val_ids in gkf.split(train_final, train_final['isFraud'], train_final['DT_split']):
    df_tr = train_final.loc[train_ids]
    df_val = train_final.loc[val_ids]
    dtrain = lgb.Dataset(df_tr[all_good_features], label=df_tr['isFraud'],
                         categorical_feature=all_good_categorical, free_raw_data=False)
    dval = lgb.Dataset(df_val[all_good_features], label=df_val['isFraud'],
                       categorical_feature=all_good_categorical, free_raw_data=False)
    
    
    model = lgb.train(params, dtrain, num_boost_round=10000, valid_sets=(dval),
                early_stopping_rounds=100, verbose_eval=50)
    preds = model.predict(test_final[all_good_features], num_iteration=model.best_iteration)
    
    all_preds += preds
    scores.append(model.best_score)

In [ ]:
scores

In [ ]:
scores

In [ ]:
model.best_score

In [ ]:
%%time
res = model.predict(shap_test[all_good_features], pred_contrib=True)
mean_shap = np.abs(res).mean(axis=0)

In [ ]:
shap_imp[shap_imp['Feature'] == 'smoothed_encoded_card1_TransactionDT_split_TransactionDT_dayOfMonth_TransactionDT_hour_on_TransactionAmt_sum']

In [ ]:
all_df[all_good_features].corr().max()

In [ ]:
shap_imp['Feature'] = shap_imp['Feature'].replace('smoothed_encoded_card1_TransactionDT_split_TransactionDT_dayOfMonth_TransactionDT_hour_on_TransactionAmt_sum', 'ROMAN_RIDICULOUSLY_LONG_FEATURE')

In [ ]:
plt.figure(figsize=(10, 10))
sns.barplot(x="Value", y="Feature", data=shap_imp.sort_values(by="Value", ascending=False)[:60])
plt.tight_layout()
plt.show()

In [ ]:
shap_imp['Feature'] = shap_imp['Feature'].replace('smoothed_encoded_card1_TransactionDT_split_TransactionDT_dayOfMonth_TransactionDT_hour_on_TransactionAmt_sum', 'ROMAN_RIDICULOUSLY_LONG_FEATURE')
plt.figure(figsize=(10, 10))
sns.barplot(x="Value", y="Feature", data=shap_imp.sort_values(by="Value", ascending=False)[:60])
plt.tight_layout()
plt.show()

In [ ]:
best_score = np.mean([score['valid_0']['auc'] for score in scores])

In [ ]:
best_score

In [ ]:
all_preds = all_preds

In [ ]:
len(all_preds)

In [ ]:
len(sub)

# Fit all (Optional)

In [ ]:
dfinal = lgb.Dataset(train_final[good_cols], label=train_final['isFraud'],
                     categorical_feature=good_categotical, free_raw_data=False)

In [ ]:
%%time
model_all = lgb.train(params, dtrain,
                      num_boost_round=model.best_iteration + int(0.1 * model.best_iteration),
                      verbose_eval=50)

# Predict

In [ ]:
%%time
preds = model.predict(test_final[good_cols], num_iteration=model.best_iteration)

In [ ]:
sub = pd.read_csv('data/sample_submission.csv')
sub['isFraud'] = all_preds
sub.to_csv(f'submissions/gkfold_12_cent_notnull{best_score:0.7}.csv.gz', index=False)

In [ ]:
f'submissions/kfold_without_te_12_folds{best_score:0.4}.csv.gz'

In [ ]:
sub2 = pd.read_csv('submissions/gkfold_d_min_max_0.94122.csv.gz')

In [ ]:
np.corrcoef(sub.isFraud, sub2.isFraud)

# Permutation Importance

# Adversarial

In [ ]:
all_df['is_test'] = [0 for _ in range(len(train))] + [1 for _ in range(len(test))]

In [ ]:
adv_df = all_df.sample(100000)

In [ ]:
good_cols = list(MODEL_FEATURES - set(['VersionNum'])) #- set(baaad))# -\
#                 set(['BrowserVersion', 'V9', 'id_31', 'id_13']))
good_categotical = list(set(CATEGORICAL_FEATURES))# - set(baaad))# -\
#                        set([ 'BrowserVersion', 'id_31', 'id_13']))\

In [ ]:
d_adv = lgb.Dataset(adv_df[good_cols], label=adv_df['is_test'], categorical_feature=good_categotical,
                    free_raw_data=False)

In [ ]:
adv_model = lgb.train(params, d_adv,
                      num_boost_round=60,
                      verbose_eval=10)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
pred = adv_model.predict(adv_df[good_cols])

In [ ]:
roc_auc_score(adv_df['is_test'], pred)

In [ ]:
['card1', 'card2', 'BrowserVersion', 'V9', 'id_31', 'id_13']
baaad = ['id_13_freq', 'D15', 'dist1', 'D11', 'dayofmonth',
         'D10', 'BrowserVersion_freq', 'id_31_freq',
         'dayofmonth_freq', 'D4', 'C12', 'V326', 'id_38',
         'M_na', 'V335', 'id_38_freq', 'V8', 'M9_freq', 'id_01', 'id_34', 'Browser',
         'id_34_freq', 'OSMajorVersion', 'OSMajorVersion_freq','card5', 'id_32', 'id_30_freq', 'id_33_freq'] +\
        [f for f in all_df.columns if f.startswith('D') and len(f) <= 3] +\
        [f for f in all_df.columns if f.startswith('V') and len(f) <= 4] +\
        [f for f in all_df.columns if f.startswith('C') and len(f) <= 4] +\
        [f for f in all_df.columns if f.startswith('id_')
         or f.startswith('uid') or f.startswith('addr')] +\
        [f for f in all_df.columns if f.startswith('M') and (len(f) <= 4 or f.endswith('freq'))]

In [ ]:
train['id_32'].hist()

In [ ]:
test['id_32'].hist()

In [ ]:
all_df[:len(train)]['V8'].hist()

In [ ]:
all_df[len(train):]['V8'].hist()

In [ ]:
len(good_cols)

In [ ]:
all_df['BrowserVersion'].value_counts()

In [ ]:
%%time
sample = adv_df.sample(5000)
res = adv_model.predict(sample[good_cols], pred_contrib=True)
mean_shap = np.abs(res).mean(axis=0)
shap_imp = pd.DataFrame(sorted(zip(mean_shap, good_cols)), columns=['Value','Feature'])
plt.figure(figsize=(10, 10))
sns.barplot(x="Value", y="Feature", data=shap_imp.sort_values(by="Value", ascending=False)[:60])
plt.tight_layout()
plt.show()

In [ ]:
from eli5.sklearn import PermutationImportance

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgb_clf = LGBMClassifier(**params)

In [ ]:
%%time
lgb_clf.fit(tr[good_cols].fillna(-1), tr['isFraud'], feature_name=good_cols, categorical_feature=good_categotical)

In [ ]:
imp = PermutationImportance(lgb_clf, 'roc_auc', refit=False)

In [ ]:
%%time
res = imp.fit(val[good_cols].fillna(-1), val['isFraud'])

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
C_cols = [col for col in all_df.columns if col.startswith('C') and len(col) < 4]

In [ ]:
C_pca = PCA(n_components=0.99).fit_transform(all_df[C_cols].fillna(0))

In [ ]:
C_pca[:, 0].shape

In [ ]:
pca_cols = []
for i, col in enumerate([f'C_PCA_{c}' for c in range(C_pca.shape[1])]):
    all_df[col] = C_pca[:, i]
    pca_cols.append(col)
    

In [ ]:
MODEL_FEATURES.update(pca_cols)

In [ ]:
V_cols = [col for col in all_df.columns if col.startswith('V') and len(col) < 5]

In [ ]:
%%time
V_pca = PCA(n_components=0.99).fit_transform(all_df[V_cols].fillna(0))

In [ ]:
v_pca_cols = []
for i, col in enumerate([f'V_PCA_{c}' for c in range(V_pca.shape[1])]):
    all_df[col] = V_pca[:, i]
    v_pca_cols.append(col)
    

In [ ]:
MODEL_FEATURES.update(v_pca_cols)

In [ ]:
diff_cols = []
d_cols = [col for col in all_df.columns if col.startswith('D') and len(col) < 3 and col != 'DT']
for col in d_cols:
    all_df[col + '_diff'] = all_df[col].diff()
    diff_cols.append(col + '_diff')

In [ ]:
g = ['id_33_freq', 'V143', 'id_11', 'id_20_freq', 'V123', 'V323', 'V267', 'uid5_D15_std', 'V269', 'id_24', 'V212', 'V234', 'uid2_D15_std', 'V24', 'V259', 'V316', 'V12', 'V334', 'V115', 'V261', 'V77', 'M6', 'C8', 'V57', 'uid3_D15_std', 'V92', 'id_28', 'V67', 'V34', 'R_emaildomain_2', 'V228', 'C12', 'V4', 'V328', 'V248', 'DeviceType_freq', 'C6', 'V294', 'V106', 'V265', 'id_12_freq', 'V139', 'V250', 'V258', 'V39', 'V18', 'V32', 'V48', 'id_08', 'addr1', 'V80', 'id_02', 'V320', 'M6_freq', 'id_23', 'uid5_TransactionAmt_std', 'OSMajorVersion_freq', 'V111', 'V10', 'P_emaildomain_1_freq', 'V225', 'R_emaildomain_3_freq', 'V199', 'D9', 'V244', 'M7', 'V50', 'OSMajorVersion', 'id_12', 'id_37_freq', 'V313', 'V101', 'id_38', 'V231', 'V216', 'V186', 'V220', 'V293', 'V55', 'uid3_freq', 'V170', 'V84', 'V98', 'V140', 'V256', 'DeviceInfoMajor', 'id_32_freq', 'V181', 'id_25_freq', 'V284', 'V29', 'V58', 'R_emaildomain_1', 'addr2_freq', 'V301', 'id_18', 'dayofmonth_freq', 'V63', 'V298', 'V289', 'V281', 'V255', 'M_na', 'V292', 'M3_freq', 'uid2_D15_mean', 'D12_day_min_max', 'V81', 'V300', 'dayofweek_freq', 'V270', 'uid4_TransactionAmt_std', 'V15', 'V175', 'id_32', 'id_19', 'V69', 'V178', 'V36', 'V260', 'V242', 'D14_day_min_max', 'V251', 'V20', 'DeviceInfo', 'V224', 'V86', 'id_37', 'id_35_freq', 'V156', 'V276', 'V303', 'addr1_freq', 'V253', 'V168', 'V335', 'V132', 'V79', 'V266', 'id_15_freq', 'V208', 'V108', 'V90', 'V185', 'V91', 'decimal_len', 'R_emaildomain_1_freq', 'P_emaildomain_2_freq', 'V133', 'V167', 'C7', 'id_10', 'V271', 'V213', 'V198', 'id_05', 'DeviceInfo_freq', 'V94', 'V126', 'M8_freq', 'V46', 'id_17', 'V155', 'V262', 'id_01', 'V53', 'V273', 'uid4_D15_mean', 'M4', 'V17', 'id_22', 'V286', 'V200', 'V246', 'id_17_freq', 'C10', 'M8', 'id_31_freq', 'V161', 'card5', 'V218', 'V26', 'V162', 'V338', 'V307', 'R_emaildomain', 'V5', 'decimal_value_freq', 'uid3_D15_mean', 'uid5_C13_mean', 'id_16_freq', 'V254', 'is_holiday_freq', 'V183', 'P_emaildomain_2', 'V76', 'P_emaildomain', 'V188', 'V195', 'V124', 'V75', 'dayofmonth', 'V95', 'D6_day_min_max', 'V35', 'V308', 'uid2_unique_amt', 'id_30', 'V336', 'V193', 'R_emaildomain_2_freq', 'V2', 'V127', 'uid4_freq', 'V45', 'V33', 'M1', 'uid2_TransactionAmt_std', 'weekofmonth', 'V73', 'id_28_freq', 'C1', 'V99', 'DeviceInfoTop', 'V243', 'V215', 'V104', 'uid4_C13_mean', 'card4_freq', 'id_34', 'V222', 'dist2', 'V83', 'M9', 'V49', 'OSName_freq', 'M3', 'V169', 'id_30_freq', 'V235', 'V311', 'D7_day_min_max', 'V205', 'V103', 'V209', 'V203', 'id_36', 'V149', 'D11_day_min_max', 'V290', 'V166', 'M9_freq', 'V236', 'V112', 'id_19_freq', 'V116', 'V31', 'id_14', 'V291', 'uid5_TransactionAmt_mean', 'id_38_freq', 'C13', 'V56', 'IsMobile_freq', 'V330', 'V85', 'V128', 'V152', 'id_36_freq', 'V11', 'V22', 'uid1_TransactionAmt_mean', 'V304', 'M5_freq', 'V25', 'V190', 'card4', 'V54', 'V109', 'P_emaildomain_1', 'id_13', 'Browser_freq', 'V43', 'V202', 'is_holiday', 'V174', 'V191', 'V219', 'V277', 'id_20', 'V23', 'id_23_freq', 'V142', 'id_09', 'D4_day_min_max', 'V157', 'V146', 'V275', 'Browser', 'V135', 'id_31', 'V263', 'V9', 'P_emaildomain_freq', 'V221', 'uid1_D15_mean', 'C5', 'V288', 'C3', 'V37', 'V187', 'D3_day_min_max', 'V165', 'V296', 'V72', 'V299', 'card3_freq', 'V147', 'D2_day_min_max', 'V272', 'V171', 'V52', 'V38', 'uid2_freq', 'V138', 'V238', 'ProductCD_freq', 'V110', 'D8_day_min_max', 'card6_freq', 'id_15', 'V264', 'C9', 'V322', 'uid1_freq', 'uid2_C13_mean', 'id_13_freq', 'V283', 'V237', 'V159', 'V227', 'V19', 'V62', 'V130', 'id_06', 'V207', 'R_emaildomain_3', 'uid4_D15_std', 'V47', 'V158', 'V282', 'V180', 'V102', 'V154', 'V206', 'V339', 'uid5_unique_amt', 'V229', 'IsMobile', 'uid4_unique_amt', 'V136', 'V100', 'uid1_C13_std', 'V177', 'V66', 'uid1_unique_amt', 'V201', 'V105', 'TransactionAmt', 'V214', 'id_07', 'V257', 'V332', 'V319', 'M4_freq', 'V114', 'C14', 'V134', 'V249', 'id_14_freq', 'V8', 'V51', 'D13_day_min_max', 'card6', 'V315', 'hour_freq', 'V245', 'V61', 'V268', 'V239', 'DeviceInfoMajor_freq', 'DeviceInfoTop_freq', 'V42', 'V148', 'V297', 'V192', 'dayofweek', 'V247', 'V204', 'id_24_freq', 'C11', 'V196', 'uid5_freq', 'V223', 'uid1_C13_mean', 'card2_freq', 'V78', 'V64', 'V295', 'uid1_D15_std', 'V172', 'V333', 'uid4_TransactionAmt_mean', 'V194', 'id_16', 'V280', 'V232', 'uid3_C13_std', 'V189', 'V324', 'V176', 'V321', 'D10_day_min_max', 'V70', 'V306', 'ProductCD', 'card5_freq', 'V60', 'V164', 'V151', 'V44', 'V337', 'V326', 'V13', 'V331', 'V317', 'V30', 'V278', 'OSName', 'id_34_freq', 'id_03', 'card1_freq', 'V7', 'V121', 'uid3_TransactionAmt_mean', 'V71', 'V226', 'M2', 'V302', 'uid3_unique_amt', 'card3', 'V217', 'V150', 'V96', 'V40', 'C2', 'weekofmonth_freq', 'V184', 'C4', 'M5', 'V145', 'V309', 'V173', 'uid5_D15_mean', 'V137', 'V210', 'decimal_value', 'uid5_C13_std', 'V182', 'V87', 'D1_day_min_max', 'V125', 'V252', 'V287', 'V160', 'D15_day_min_max', 'V211', 'V310', 'uid2_C13_std', 'id_26_freq', 'M7_freq', 'V163', 'V312', 'uid1_TransactionAmt_std', 'V131', 'V329', 'R_emaildomain_freq', 'uid2_TransactionAmt_mean', 'V285', 'V274', 'V129', 'V230', 'V74', 'uid4_C13_std', 'V279', 'V3', 'V59', 'dist1', 'uid3_C13_mean', 'V82', 'uid3_TransactionAmt_std', 'V233', 'id_33', 'V144', 'M2_freq', 'V197', 'id_18_freq', 'V153', 'V6', 'V314', 'V97', 'V318', 'V179', 'id_04', 'id_21_freq', 'D5_day_min_max', 'DeviceType', 'V93']

In [ ]:
len(g)